In [ ]:
!pip install --upgrade "watson-developer-cloud>=2.0.1"

In [ ]:
from watson_developer_cloud import SpeechToTextV1

# Change the STT
speech_to_text = SpeechToTextV1(
    username='{Your_STT_Service_Username}',
    password='{Your_STT_Service_Password}',
    url='{Your_STT_Service_URL}'
)

In [ ]:
import json
speech_model = speech_to_text.get_model('en-US_NarrowbandModel').get_result()
print(json.dumps(speech_model, indent=2))

In [ ]:
from os.path import join, dirname

files = ['{Your_Audio_File.wav}']
for file in files:
    with open(join(dirname('.'), './.', file),
                   'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/wav', #change this if it is not a .wav file
            timestamps=True,
            word_alternatives_threshold=0.9,
            keywords=['IoT', 'network', 'sensors', 'internet', 'devices'], #change this to your keywords
            keywords_threshold=0.5).get_result()
    print(json.dumps(speech_recognition_results, indent=2))

In [ ]:
# Create a custom Language model
language_model = speech_to_text.create_language_model(
    'First example language model',
    'en-US_NarrowbandModel',
    description='First custom language model example').get_result()
print(json.dumps(language_model, indent=2))

speech_to_text.delete_language_model(language_model['customization_id'])

In [ ]:
# print your custom language model's ID
language_models = speech_to_text.list_language_models().get_result()
print(json.dumps(language_models, indent=2))

In [ ]:
custom_lang_narrowband_model_id = '{your_custom_lang_id}'

In [ ]:
speech_to_text.upgrade_language_model(custom_lang_narrowband_model_id)

In [ ]:
with open(join(dirname('.'), './.', 'corpora.txt'), #modify corpora.txt to your file name
               'rb') as corpus_file:
    speech_to_text.add_corpus(
        custom_lang_narrowband_model_id,
        'corpus1', # you can name it however you want 
        corpus_file)

In [ ]:
corpora = speech_to_text.list_corpora(custom_lang_narrowband_model_id).get_result()
print(json.dumps(corpora, indent=2))

In [ ]:
words = speech_to_text.list_words(custom_lang_narrowband_model_id).get_result()
print(json.dumps(words, indent=2))

In [ ]:
from watson_developer_cloud.speech_to_text_v1 import CustomWord

# just an examle demonstrating the creation of a custom word
ibmdotxt = CustomWord (
    'ibm.txt',
    ['IBM dot text'])

speech_to_text.add_words(
    custom_lang_narrowband_model_id,
    [ibmdotxt])

In [ ]:
# Training takes up to a minute
speech_to_text.train_language_model(custom_lang_narrowband_model_id,'all')

In [ ]:
# You can check the status of the language model - wait until it is 'available'
import time

language_models = speech_to_text.list_language_models().get_result()
models = language_models["customizations"]

statusNotAvailable = True
while statusNotAvailable:    
    for model in models:
        if model['customization_id'] == custom_lang_narrowband_model_id: 
            if model['status'] == 'available':
                statusNotAvailable = False
                break
            else:
                time.sleep(5)
            print(model['status'])

In [ ]:
acoustic_models = speech_to_text.list_acoustic_models('en-US').get_result()
print(json.dumps(acoustic_models, indent=2))

In [ ]:
custom_acoustic_narrowband_model_id = '{Your_Acoustic_Model_ID}'
audio_resources = speech_to_text.list_audio(custom_acoustic_narrowband_model_id).get_result()
print(json.dumps(audio_resources, indent=2))

In [ ]:
with open(join(dirname('.'), '.', '{YourTrainingAudioFile}'),
               'rb') as audio_file:
    speech_to_text.add_audio(
        custom_acoustic_narrowband_model_id,
        'audio1', #you can set this to whatever value
        audio_file,
        'audio/wav' # change this as per file type
    )

In [ ]:
speech_to_text.upgrade_acoustic_model(custom_acoustic_narrowband_model_id)

In [ ]:
speech_to_text.train_acoustic_model(custom_acoustic_narrowband_model_id,custom_lang_narrowband_model_id)

In [ ]:
# Get status of the language model - wait until it is 'available'
acoustic_models = speech_to_text.list_acoustic_models('en-US').get_result()
models = acoustic_models["customizations"]

statusNotAvailable = True
while statusNotAvailable:    
    #print(json.dumps(language_models, indent=2))
    for model in models:
        if model['customization_id'] == custom_acoustic_narrowband_model_id: 
            if model['status'] == 'available':
                print(model['status'])
                statusNotAvailable = False
                break
            else:
                time.sleep(15)
                print(model['status'])

In [ ]:
files = ['{Your_Audio_File}']
for file in files:
    with open(join(dirname('.'), './.', file),
                   'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(
            audio=audio_file,
            content_type='audio/wav',
            model='en-US_NarrowbandModel',
            customization_id=custom_lang_narrowband_model_id,
            acoustic_customization_id=custom_acoustic_narrowband_model_id,
            timestamps=True,
            word_alternatives_threshold=0.9,
            keywords=['IoT', 'network', 'sensors', 'internet', 'devices'],
            keywords_threshold=0.5).get_result()
    print(json.dumps(speech_recognition_results, indent=2))